# simulation with ivs data

### Imports
Import the required libraries

In [1]:
# package(s) related to time, space and id
import datetime, time
import platform
import random
import os
import pathlib

# you need these dependencies (you can get these from anaconda)
# package(s) related to the simulation
import simpy

# spatial libraries 
import pyproj
import shapely.geometry
from shapely.geometry import Point
import shapely
import geopandas as gpd
import movingpandas as mpd

# package(s) for data handling
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

# OpenTNSIM
import opentnsim
import opentnsim.core as core
import opentnsim.graph_module as graph_module
import opentnsim.plot as plot


# dtv_backend
import dtv_backend.fis as fis
import dtv_backend.network
import dtv_backend.network.network_utilities
import dtv_backend.postprocessing
import dtv_backend.simple
import dtv_backend.simulate

# Used for mathematical functions
import math             
import numpy as np

# Used for making the graph to visualize our problem
import networkx as nx  
import pickle

### Create graph

In [2]:
graph1 =pickle.load(open("dtv_backend\data\euris\euris_graph_v0.1.pickle", "rb"))
graph = graph_module.Graph()
graph.graph = graph1
graph.graph_info = opentnsim.utils.info(graph.graph)

In [3]:
df_edges = nx.to_pandas_edgelist(graph1)
gpd_edges = gpd.GeoDataFrame(df_edges[['source', 'target', 'name', 'mlengthcm', 'cemt', 'geometry', 'mwidthcm']])
gpd_edges['target'] = gpd_edges.target.str.join("_") # shape file cannot handle tuples
gpd_edges['source'] = gpd_edges.source.str.join("_") # shape file cannot handle tuples

gpd_edges.to_file("dtv_backend/plots_routes/edges_zonder_afzetting")

### Make vessels and paths

##### read data

In [4]:
data = gpd.read_file("dtv_backend\data\ivs\ivs-2024-geocoded-sample.gpkg")
data['datetime'] = pd.to_datetime(data['v05_06_begindt_evenement_iso'], format = 'ISO8601', errors = 'coerce')
data.dropna(subset = ['datetime', 'geometry'], inplace = True)
data1=data

D:\Users\ALPF\AppData\Local\Temp\ipykernel_19160\2354379436.py:2: FutureWarning:

In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`



filter only on rotterdam to duisburg, 2023. Only keep 5 ships

In [5]:
data = data1
data = data[
    (data.UNLO_herkomst== "NLDOR")
    #(data.UNLO_herkomst== "NLGOR")
     & (data.UNLO_bestemming.str.startswith("DE"))
     & (data.v05_06_begindt_evenement_iso.str.startswith("2023"))
     ].head()
data

,Jaarmaand,Jaar,Maand,Weeknr,v05_06_begindt_evenement_iso,v05_06_Begindt_evenement,UNLO_herkomst,UNLO_bestemming,v15_1_Scheepstype_RWS,SK_CODE,v18_Laadvermogen,v28_Beladingscode,v38_Vervoerd_gewicht,v30_4_Containers_TEU_S,nstr_nw,nst2007_nw,geometry,datetime
14694,2308,2023,8,32,2023-08-10T08:00:00+02:00,10 augustus 2023 08:00:00 uur,NLDOR,DEDUI,9,C3l,4320.0,5.0,0.0,0.0,9,16,"LINESTRING (4.63333 51.78333, 6.75000 51.46667)",2023-08-10 08:00:00+02:00
20814,2301,2023,1,1,2023-01-04T16:00:00+01:00,04 januari 2023 16:00:00 uur,NLDOR,DEMHG,9,C3l,5730.0,7.0,3550000.0,0.0,2,2.1,"LINESTRING (4.63333 51.78333, 8.45000 49.48333)",2023-01-04 16:00:00+01:00
27391,2309,2023,9,39,2023-09-25T10:00:00+02:00,25 september 2023 10:00:00 uur,NLDOR,DENSS,2,M3,690.0,5.0,0.0,0.0,9,16,"LINESTRING (4.63333 51.78333, 6.68333 51.18333)",2023-09-25 10:00:00+02:00
32727,2309,2023,9,39,2023-09-26T06:00:00+02:00,26 september 2023 06:00:00 uur,NLDOR,DEGEK,1,M2,510.0,7.0,375000.0,0.0,0,1.1,"LINESTRING (4.63333 51.78333, 7.11667 51.51667)",2023-09-26 06:00:00+02:00
35894,2308,2023,8,32,2023-08-12T15:00:00+02:00,12 augustus 2023 15:00:00 uur,NLDOR,DEFRA,21,BII-2L,5320.0,5.0,0.0,0.0,9,16,"LINESTRING (4.63333 51.78333, 8.68333 50.11667)",2023-08-12 15:00:00+02:00


#### Create vessels

In [6]:
# Make a class out of mix-ins
TransportResource = type('TransportResource', 
                         (core.Identifiable, core.ContainerDependentMovable, 
                          core.HasResource, core.Routable,
                          core.VesselProperties,
                         core.ExtraMetadata), 
                         {})

#define speed: 
def compute_v_provider(v_empty, v_full):
    return lambda x: 1

vessels = []
for index, row in data.iterrows():
    #determine path
    point_1 = fis.find_closest_node(graph.graph, Point(row.geometry.coords[0]))
    point_2 = fis.find_closest_node(graph.graph, Point(row.geometry.coords[-1]))
    path = nx.dijkstra_path(graph.graph, point_1[0], point_2[0], weight = 'length_m')
    #determine capacity
    capacity = max(row.v18_Laadvermogen*1000, row.v38_Vervoerd_gewicht, 1)
    data_vessel = {"env": None,
               "name": row.name,
               "type": row['v15_1_Scheepstype_RWS'],
               "B": 1,
               "L": 10,
               "route": path,
               "geometry": Point(row.geometry.coords[0]),  # lon, lat
               "capacity": capacity,
               "v": 1,
               "compute_v": compute_v_provider(v_empty=1, v_full=1),
               }
    vessel = TransportResource(**data_vessel)
    vessels.append(vessel)
    break

# vessels = [TransportResource(**{
#     "env" : None,
#     "name": 'korte_route', 
#     "type": 'M6',
#     "B": 1, 
#     "L": 10,
#     "route": nx.dijkstra_path(graph.graph, "8865735", "8861687"),
#     "geometry": Point(row.geometry.coords[0]),  # lon, lat
#     "capacity": capacity,
#     "v": 1,
#     "compute_v": compute_v_provider(v_empty=1, v_full=1),
# })]

### Define general outline of env and simulation

In [7]:
def start(env, vessel):
    while True:
        vessel.log_entry_v0("Start sailing", env.now, "", vessel.geometry)
        yield from vessel.move()
        vessel.log_entry_v0("Stop sailing", env.now, "", vessel.geometry)
        
        if vessel.geometry == nx.get_node_attributes(env.FG, "geometry")[vessel.route[-1]]:
            break

In [8]:
# Start simpy environment
simulation_start = datetime.datetime.now()
env = simpy.Environment(initial_time = time.mktime(simulation_start.timetuple()))
env.epoch = time.mktime(simulation_start.timetuple())


# Add graph to environment
graph.add_resources(list(graph.graph.edges), np.ones(len(list(graph.graph.edges))), env)



In [9]:
vessels = vessels
for i, vessel in enumerate(vessels):
    # Add environment and path to the vessel
    vessel.env = env

    # Add the movements of the vessel to the simulation
    env.process(start(env, vessel))




### Run code for original graph

In [10]:
env.FG = graph.graph.copy()
env.epoch = datetime.datetime.now()

env.run()

##### Obtain vessel log information
The cel below uses the vessel log. The core function *log_entry* is used, which takes four arguments:

- **Log.** A text to describe what is logged.
- **t.** The timestamp.
- **Value.**  The value for the log (for sailing this is the distance).
- **Geometry** The location of the vessel while loggin.

In [11]:
vessel_log = gpd.GeoDataFrame(vessels[0].logbook, geometry='Geometry')
vessel_log = mpd.Trajectory(vessel_log, traj_id='vessel_1_trip_1', obj_id='vessel_1', t='Timestamp')
#vessel_log.to_line_gdf()
vessel_log.to_line_gdf().to_file('dtv_backend/plots_routes/original_graph_logboek_movingpandas.gpkg')

d:\Users\ALPF\AppData\Local\pypoetry\Cache\virtualenvs\dtv-backend-pvs5eNot-py3.10\lib\site-packages\movingpandas\trajectory.py:166: MissingCRSWarning:

Trajectory generated without CRS. Computations will use Euclidean distances.



### Run code for adapted graph

### Visualization of path
If you get an error regarding ffmpeg use [this](https://stackoverflow.com/questions/13316397/matplotlib-animation-no-moviewriters-available) answer. You have to install ffmpeg in your Conda environment. It can be done using the following command.

```bash
#Install ffmpeg using Conda
conda install -c conda-forge ffmpeg
```

In [12]:
# # visualise vessel movements based on the information included in the vessel.log
# plot.vessel_kml(env, vessels, stepsize = 60)
# plot.graph_kml(env)
